# Exercice Capstone – Company Sales

## Objectifs
- Lire deux CSV : `sales_data.csv` et `employee_data.csv`.
- Calculer les ventes **par employé** puis **par département**.
- Générer un rapport `report.csv` contenant `department,total_sales`.

---

### Consignes
1. Lire le fichier `sales_data.csv` et agréger les ventes par employé.
2. Lire le fichier `employee_data.csv` et créer un dictionnaire `employee_id → department`.
3. Associer ventes et départements, puis agréger les ventes par département.
4. Écrire un fichier `report.csv` contenant `department,total_sales`.
5. (Bonus) Trier les départements par ventes décroissantes et écrire `report_sorted.csv`.

⚠️ La gestion des erreurs sera traitée **dans la séance suivante (Exceptions)**.

### Imports & chemins

In [2]:
from pathlib import Path
import csv

# TODO: définir les chemins des fichiers
SALES_FILE = Path("./files/sales_data.csv")
EMPLOYEES_FILE = Path("./files/employee_data.csv")
REPORT_FILE = Path("./files/report.csv")

print("Files existants:")
print("- Sales_data.csv:", SALES_FILE.exists())
print("- Employee_data.csv:", EMPLOYEES_FILE.exists())


Files existants:
- Sales_data.csv: True
- Employee_data.csv: True


### Lecture des CSV

In [3]:
# TODO: implémenter une fonction read_csv_rows(path: Path) -> list[dict]
# Utiliser csv.DictReader
def read_csv_rows(path: Path) -> list[dict]:
    """
    Lecture simple d'un CSV en liste de dictionnaires (suppose les données sont valides).
    :param path: Path
    :return: List[dict]
    """
    with open(path, encoding='utf-8') as file:
        reader = csv.DictReader(file)
        return list(reader)

sales_rows = read_csv_rows(SALES_FILE)
employee_rows = read_csv_rows(EMPLOYEES_FILE)

print("Exemples de lignes:")
len(sales_rows), len(employee_rows)

Exemples de lignes:


(100, 10)

### Ventes par employé

In [ ]:
# TODO: implémenter aggregate_sales_by_employee(rows: list[dict]) -> dict[str, float]
# Agréger les montants par employee_id
def aggregate_sales_by_employee(rows: list[dict]) -> dict[str, float]:
    """
    Agrège les montants des ventes par employé, employee_id (lecture simple)
    :param rows: list[dict]
    :return: dict[str, float]
    """
    totals: dict[str, float] = {}

    for row in rows:
        emp_id = row["EmployeeID"]
        amount = float(row['Amount'])
        totals[emp_id] = totals.get(emp_id, 0.0) + amount

    return totals

# print("Colonnes des ventes:", sales_rows[0].keys())

sales_by_employee = aggregate_sales_by_employee(sales_rows)
print("Exemple de ventes par employé:", list(sales_by_employee.items())[:5])


### Index employé → département

In [ ]:
# TODO: implémenter build_employee_department_index(rows: list[dict]) -> dict[str, str]
# Construire un dictionnaire {employee_id: department}
def build_employee_department_index(rows: list[dict]) -> dict[str, str]:
    """
    Construit un index employee_id -> department (lecture simple)
    :param rows: list[dict]
    :return: dict[str, str]
    """
    idx: dict[str, str] = {}

    for row in rows:
        idx[row["EmployeeID"]] = row["Department"]
    return idx

emp_to_dept = build_employee_department_index(employee_rows)
list(emp_to_dept.items())[:5]

### Ventes par département

In [ ]:
# TODO: implémenter aggregate_sales_by_department(sales_by_emp: dict[str, float], emp_to_dept: dict[str, str]) -> dict[str, float]
# Agréger les ventes par département
def aggregate_sales_by_department(sales_by_employee: dict[str, float], emp_to_dept: dict[str, str]) -> dict[str, float]:
    """
    Jointure simple employee_id -> department, puis agrégation par department.
    Employées sans department sont ignorées, dans cette version simple.
    :param sales_by_employee: dict[str, float]
    :param emp_to_dept: dict[str, str]
    :return: dict[str, float]
    """
    totals: dict[str, float] = {}

    for emp_id, total in sales_by_employee.items():
        dep = emp_to_dept.get(emp_id)
        if dep is None:
            continue  # Employée sans département, on ignore
        totals[dep] = totals.get(dep, 0.0) + total
    return totals

sales_by_department = aggregate_sales_by_department(sales_by_employee, emp_to_dept)
sales_by_department

### Écriture du rapport

In [ ]:
# TODO: implémenter write_report(path: Path, dept_totals: dict[str, float])
# Écrire un CSV department,total_sales
def write_report(path: Path, dept_totals: dict[str, float]) -> None:
    """
    Écrit un rapport CSV simple department, total_sales (formatée à 2 décimales)
    :param path: Path
    :param dept_totals: dict[str, float]
    """
    with open(path, 'w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["department", "total_sales"])
        for dept, total in dept_totals.items():
            writer.writerow([dept, f"{total:.2f}"])  # Formatage à 2 décimales

write_report(REPORT_FILE, sales_by_department)
REPORT_FILE.resolve()

### Vérifications

In [ ]:
# TODO: vérifier que report.csv existe et contient au moins un header + 1 ligne
assert REPORT_FILE.exists(), "Le fichier report.csv n'existe pas"

with open(REPORT_FILE, encoding='utf-8') as file:
    lines = file.readlines()

assert len(lines) >= 2, "Le fichier report.csv ne contient pas de données"
print("report.csv contient", len(lines)-1, "lignes de données")

## Bonus
- Trier les départements par ventes décroissantes.
- Sauvegarder dans un fichier `report_sorted.csv`.
- (Optionnel) Si matplotlib est installé, tracer un bar chart des ventes.


In [ ]:
# Bonus : trier par ventes décroissantes et écrire report_sorted.csv
def sorted_totals_desc(deps_totals: dict[str, float]) -> list[tuple[str, float]]:
    """
    Trie les départements par ventes décroissantes.
    :param deps_totals: dict[str, float]
    :return: list[tuple[str, float]]
    """
    return sorted(deps_totals.items(), key=lambda item: item[1], reverse=True)

sorted_depts = sorted_totals_desc(sales_by_department)

REPORT_SORTED_FILE = Path("./files/report_sorted.csv")
with open(REPORT_SORTED_FILE, 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["department", "total_sales"])
    for dept, total in sorted_depts:
        writer.writerow([dept, f"{total:.2f}"])  # Formatage à 2 décimales

print("Rapport trié : ", REPORT_SORTED_FILE.resolve())
sorted_depts[:5]  # Top 5 départements